In [1]:
import os
cwd = os.getcwd()
pygad_loc = '/grid/home/nbourgeois/codonOpt'
os.chdir(pygad_loc)
from general_functions import *
from metrics import *
os.chdir(cwd)

Using TensorFlow backend.


In [2]:
ga_input = '/grid/home/nbourgeois/data/codon_jason/final_seq.fa'
output = '' #output filename
tissue = 'Liver'

In [3]:
def get_codon_weights(loc):
    # read table
    try:
        index_loc = os.path.join(pygad_loc,loc)
        codon_table_raw = pd.read_csv(index_loc,sep='\t')

    except FileNotFoundError:
        print('Invalid Tissue')
        return()    

    codon_table = codon_table_raw.set_index('#Codon')
    maxes = codon_table_raw.groupby(by=['AA'])['Frequency'].max()
    codon_table['max_col'] = codon_table.apply(lambda x: maxes[x['AA']],axis=1)
    codon_table['weight'] = codon_table['Frequency'] / codon_table['max_col']
    weight_dict = codon_table['weight'].to_dict()
    
    return(weight_dict)

In [4]:
(fastaIDs, fastaSeqs) = readFasta(ga_input)
names = ['_'.join(anid.split(':')[1].split('_')[0:2]) for anid in fastaIDs]

# CAI

In [5]:
orig_weights = get_codon_weights('references/CoCoPUTs_codon_usage/codon_usage/Liver.codon.txt')
# bai_weight_dict = get_bicodon_weights(tissue)

In [6]:
new_weights = get_codon_weights('tissue_cocoputs/data/Test_liver.codon.txt')
hi_weights = get_codon_weights('tissue_cocoputs/data/Test_liver_high.codon.txt')



In [7]:
orig_cais = []
new_cais = []
hi_cais = []
diffs = []
hidiffs = []
for fid,aa_seq in zip(fastaIDs, fastaSeqs):
    orig_cai = get_cai(aa_seq, orig_weights) *100
    orig_cais.append(orig_cai)
    new_cai = get_cai(aa_seq, new_weights)*100
    new_cais.append(new_cai)
    hi_cai = get_cai(aa_seq, hi_weights)*100
    hi_cais.append(hi_cai)
    diffs.append(new_cai - orig_cai)
    hidiffs.append(hi_cai - orig_cai)


In [8]:
pd.DataFrame(data = {'Name':names,'Orig':orig_cais,'New':new_cais,'Diff':diffs,'High':hi_cais,'Hi-Diff':hidiffs})

,Name,Orig,New,Diff,High,Hi-Diff
0,cai_0,83.963608,83.890579,-0.073029,80.210590,-3.753018
1,cai_1,88.048398,88.262931,0.214533,85.284358,-2.764040
2,cai_2,87.765231,87.993236,0.228005,84.871991,-2.893240
3,cpg_0,66.546848,67.038484,0.491636,60.166190,-6.380658
4,cpg_1,65.014299,65.551462,0.537163,58.606760,-6.407539
5,cpg_2,68.012043,68.466356,0.454314,62.004401,-6.007642
6,bai_0,87.985176,88.135984,0.150808,84.877158,-3.108018
7,bai_1,88.680959,88.947771,0.266812,85.516224,-3.164735
8,bai_2,87.944822,88.121900,0.177078,84.480205,-3.464617
9,cpg-n_0,59.220453,57.606388,-1.614065,53.373740,-5.846713


In [9]:
#cai

In [ ]:

for fid,aa_seq in zip(fastaIDs, fastaSeqs):
    cai = get_cai(aa_seq, cai_weight_dict)

    print("{}:\tCAI is {}".format(fid,cai))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_131766/2365681367.py", line 2, in <module>
    cai = get_cai(aa_seq, cai_weight_dict)
NameError: name 'cai_weight_dict' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, 

# BAI

In [10]:
#bai

In [11]:
for aa_seq,id in fastaIDs fastaSeqs:
    bai = get_bai(aa_seq, bai_weight_dict)
    print("{}:\tBAI is {}".format(id,bai))

ERROR! Session/line number was not unique in database. History logging moved to new session 2789


SyntaxError: invalid syntax (1939985177.py, line 1)

In [ ]:
#cpg
for aa_seq,id in fastaIDs fastaSeqs:
    bai = get_cpg(aa_seq, bai_weight_dict)
    print("{}:\tcpg is {}".format(id,bai))

In [ ]:
global codon_to_int

codon_to_int, gene_space = init_parameters_fun(aa_seq)
    
gene_space_int = [[codon_to_int[x] for x in y] for y in gene_space]